In [1]:
import pandas as pd
import numpy as np
import pydit


In [2]:
d1 = {
        "mkey": [1, 2, 3,4],
        "mvalue": ["a","b","c","d"]
    }
d2 = {
        "tkey": [100, 101, 102,103,104,105,106],
        "mkey": [1,1,1,2,3,3,5],
        "tvalue": [10.12,20.4,33.3,45,59.99,60,-1]
    }

df1 = pd.DataFrame(data=d1)
df2= pd.DataFrame(data=d2)